In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import RFE
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# ========== Loss Monitoring Functions ========== #
def calculate_loss_metrics(y_true, y_pred):
    """Calculate various loss metrics for monitoring"""
    losses = {}
    
    # Basic losses
    losses['mse'] = mean_squared_error(y_true, y_pred)
    losses['rmse'] = np.sqrt(losses['mse'])
    losses['mae'] = mean_absolute_error(y_true, y_pred)
    losses['mape'] = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100
    
    # Huber loss (robust to outliers)
    delta = 1.0
    residual = y_true - y_pred
    losses['huber'] = np.mean(np.where(np.abs(residual) <= delta,
                                      0.5 * residual**2,
                                      delta * (np.abs(residual) - 0.5 * delta)))
    
    # Log loss (suitable for non-negative predictions)
    losses['log_loss'] = np.mean((np.log(y_pred + 1) - np.log(y_true + 1))**2)
    
    return losses

def calculate_comprehensive_metrics(y_true, y_pred):
    """Calculate comprehensive evaluation metrics"""
    # Basic metrics
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    evs = explained_variance_score(y_true, y_pred)
    
    # SMAPE (Symmetric Mean Absolute Percentage Error)
    def smape(y_true, y_pred):
        numerator = np.abs(y_pred - y_true)
        denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
        mask = denominator != 0
        smape_values = np.zeros_like(numerator)
        smape_values[mask] = (numerator[mask] / denominator[mask]) * 100
        return np.mean(smape_values)
    
    smape_score = smape(y_true, y_pred)
    
    return {
        'mse': mse,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'evs': evs,
        'smape': smape_score
    }

# ========== Hyperparameter Optimization with Loss Monitoring ========== #
def fine_tune_xgb_hyperparams(X_train, y_train, X_val, y_val):
    """XGBoost hyperparameter fine-tuning with loss monitoring"""
    print("Starting XGBoost hyperparameter optimization...")
    
    param_grid = {
        'n_estimators': [90, 95, 100, 105, 110, 115],
        'max_depth': [5, 6, 7, 8],
        'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12]
    }
    
    best_score = -999
    best_params = None
    best_losses = None
    search_count = 0
    max_search = 50
    all_search_losses = []
    
    for n_est in param_grid['n_estimators']:
        for max_d in param_grid['max_depth']:
            for lr in param_grid['learning_rate']:
                if search_count >= max_search:
                    break
                    
                model = xgb.XGBRegressor(
                    n_estimators=n_est,
                    max_depth=max_d,
                    learning_rate=lr,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    reg_alpha=0.1,
                    reg_lambda=0.1,
                    random_state=42,
                    verbosity=0
                )
                
                # Train model
                model.fit(X_train, y_train)
                
                # Predict
                pred_train = model.predict(X_train)
                pred_val = model.predict(X_val)
                
                # Calculate various losses
                train_losses = calculate_loss_metrics(y_train, pred_train)
                val_losses = calculate_loss_metrics(y_val, pred_val)
                
                # R2 as main evaluation metric
                score = r2_score(y_val, pred_val)
                
                # Record search process
                search_info = {
                    'params': {
                        'n_estimators': n_est,
                        'max_depth': max_d,
                        'learning_rate': lr
                    },
                    'r2_score': score,
                    'train_losses': train_losses,
                    'val_losses': val_losses
                }
                all_search_losses.append(search_info)
                
                if score > best_score:
                    best_score = score
                    best_params = {
                        'n_estimators': n_est,
                        'max_depth': max_d,
                        'learning_rate': lr,
                        'subsample': 0.8,
                        'colsample_bytree': 0.8,
                        'reg_alpha': 0.1,
                        'reg_lambda': 0.1
                    }
                    best_losses = {
                        'train': train_losses,
                        'val': val_losses
                    }
                
                # Print progress every 10 searches
                if (search_count + 1) % 10 == 0:
                    print(f"Search progress: {search_count + 1}/{max_search}, Current best R2: {best_score:.4f}")
                
                search_count += 1
    
    print(f"XGB best parameters: {best_params}")
    print(f"XGB best score: {best_score:.4f}")
    
    # Print detailed loss information
    print(f"XGB Best Model Loss Analysis:")
    print(f"Training - MSE: {best_losses['train']['mse']:.4f}, RMSE: {best_losses['train']['rmse']:.4f}, MAE: {best_losses['train']['mae']:.4f}")
    print(f"Validation - MSE: {best_losses['val']['mse']:.4f}, RMSE: {best_losses['val']['rmse']:.4f}, MAE: {best_losses['val']['mae']:.4f}")
    print(f"Overfitting check - RMSE difference: {best_losses['val']['rmse'] - best_losses['train']['rmse']:+.4f}")
    
    return best_params, best_losses, all_search_losses

def fine_tune_lgb_hyperparams(X_train, y_train, X_val, y_val):
    print("Starting LightGBM hyperparameter optimization...")
    
    param_combinations = [
        {'n_estimators': 90, 'max_depth': 5, 'learning_rate': 0.09},
        {'n_estimators': 95, 'max_depth': 6, 'learning_rate': 0.1},
        {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1},
        {'n_estimators': 105, 'max_depth': 6, 'learning_rate': 0.11},
        {'n_estimators': 110, 'max_depth': 7, 'learning_rate': 0.1},
        {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.12},
    ]
    
    best_score = -999
    best_params = None
    best_losses = None
    all_lgb_losses = []
    
    for i, params in enumerate(param_combinations):
        model = lgb.LGBMRegressor(
            **params,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=0.1,
            random_state=42,
            verbosity=-1
        )
        
        # Train model
        model.fit(X_train, y_train)
        
        # Predict
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)
        
        # Calculate various losses
        train_losses = calculate_loss_metrics(y_train, pred_train)
        val_losses = calculate_loss_metrics(y_val, pred_val)
        
        # R2 as main evaluation metric
        score = r2_score(y_val, pred_val)
        
        # Record search process
        search_info = {
            'params': params,
            'r2_score': score,
            'train_losses': train_losses,
            'val_losses': val_losses
        }
        all_lgb_losses.append(search_info)
        
        if score > best_score:
            best_score = score
            best_params = params
            best_losses = {
                'train': train_losses,
                'val': val_losses
            }
        
        print(f"LGB combination {i+1}/{len(param_combinations)}: R2 = {score:.4f}, "
              f"Val RMSE = {val_losses['rmse']:.4f}")
    
    best_params.update({
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1
    })
    
    print(f"LGB best parameters: {best_params}")
    print(f"LGB best score: {best_score:.4f}")
    
    # Print detailed loss information
    print(f"LGB Best Model Loss Analysis:")
    print(f"Training - MSE: {best_losses['train']['mse']:.4f}, RMSE: {best_losses['train']['rmse']:.4f}, MAE: {best_losses['train']['mae']:.4f}")
    print(f"Validation - MSE: {best_losses['val']['mse']:.4f}, RMSE: {best_losses['val']['rmse']:.4f}, MAE: {best_losses['val']['mae']:.4f}")
    print(f"Overfitting check - RMSE difference: {best_losses['val']['rmse'] - best_losses['train']['rmse']:+.4f}")
    
    return best_params, best_losses, all_lgb_losses

def ultra_fine_weight_search(models_predictions, y_true):
    print("Starting CORRECTED dual model weight search...")
    
    xgb_range = np.arange(0.70, 0.90, 0.01)  # Expanded range
    
    best_score = -999
    best_config = None
    search_count = 0
    weight_search_results = []
    
    for xgb_w in xgb_range:
        lgb_w = 1.0 - xgb_w  # Ensure weights sum to 1
        
        weights = np.array([xgb_w, lgb_w])
        
        # Two-model ensemble
        ensemble_pred = np.average(models_predictions, axis=0, weights=weights)
        
        # Calculate multiple losses
        losses = calculate_loss_metrics(y_true, ensemble_pred)
        score = r2_score(y_true, ensemble_pred)
        
        # Record weight search results
        weight_result = {
            'weights': weights.copy(),
            'xgb_weight': xgb_w,
            'lgb_weight': lgb_w,
            'r2_score': score,
            'losses': losses
        }
        weight_search_results.append(weight_result)
        
        if score > best_score:
            best_score = score
            best_config = weight_result.copy()
        
        search_count += 1
    
    print(f"Searched {search_count} weight combinations")
    print(f"Best weights: XGB={best_config['xgb_weight']:.3f}, LGB={best_config['lgb_weight']:.3f}")
    print(f"Weight sum verification: {best_config['xgb_weight'] + best_config['lgb_weight']:.6f}")
    print(f"Best R2: {best_config['r2_score']:.4f}")
    
    # Print detailed loss for best weight combination
    print(f"Best Weight Combination Loss Analysis:")
    best_losses = best_config['losses']
    print(f"MSE: {best_losses['mse']:.4f}, RMSE: {best_losses['rmse']:.4f}")
    print(f"MAE: {best_losses['mae']:.4f}, MAPE: {best_losses['mape']:.2f}%")
    print(f"Huber Loss: {best_losses['huber']:.4f}")
    
    return best_config, weight_search_results

def advanced_postprocessing(y_pred, y_true, facies, porosity, permeability):
    """Advanced post-processing with geological constraints"""
    y_pred_processed = y_pred.copy()
    
    print(f"Before post-processing: range=[{y_pred.min():.2f}, {y_pred.max():.2f}], mean={y_pred.mean():.2f}")
    
    #  Geological constraints
    constraint_stats = {'mud_dry': 0, 'low_poro': 0, 'low_perm': 0, 'layer_type': 0}
    
    for i in range(len(y_pred_processed)):
        # Absolute constraints
        if facies.iloc[i] in ['MudLayer', 'DryLayer']:
            y_pred_processed[i] = 0
            constraint_stats['mud_dry'] += 1
            continue
        
        # Adaptive constraints
        poro_threshold = np.percentile(porosity, 10)
        perm_threshold = np.percentile(permeability, 15)
        
        if porosity.iloc[i] < poro_threshold and y_pred_processed[i] > 20:
            y_pred_processed[i] = min(y_pred_processed[i], 20)
            constraint_stats['low_poro'] += 1
        
        if permeability.iloc[i] < perm_threshold and y_pred_processed[i] > 12:
            y_pred_processed[i] = min(y_pred_processed[i], 12)
            constraint_stats['low_perm'] += 1
        
        # Layer type constraints
        if facies.iloc[i] == 'LowOilLayer' and y_pred_processed[i] > 75:
            y_pred_processed[i] = min(y_pred_processed[i], 75)
            constraint_stats['layer_type'] += 1
        elif facies.iloc[i] == 'OilLayer' and y_pred_processed[i] > 280:
            y_pred_processed[i] = min(y_pred_processed[i], 280)
            constraint_stats['layer_type'] += 1
    
    # Outlier detection and correction
    relative_error = np.abs(y_pred_processed - y_true) / (np.abs(y_true) + 1e-8)
    
    # Dynamic thresholds
    error_q75 = np.percentile(relative_error, 75)
    error_q90 = np.percentile(relative_error, 90)
    error_q95 = np.percentile(relative_error, 95)
    
    fatal_threshold = max(8.0, error_q95 * 1.2)
    extreme_threshold = max(4.0, error_q90 * 1.1)
    severe_threshold = max(2.5, error_q75 * 1.5)
    moderate_threshold = max(1.2, error_q75)
    
    # Five-level correction strategy
    fatal_mask = relative_error > fatal_threshold
    extreme_mask = (relative_error > extreme_threshold) & (relative_error <= fatal_threshold)
    severe_mask = (relative_error > severe_threshold) & (relative_error <= extreme_threshold)
    moderate_mask = (relative_error > moderate_threshold) & (relative_error <= severe_threshold)
    mild_mask = (relative_error > 0.8) & (relative_error <= moderate_threshold)
    
    correction_stats = {}
    
    if fatal_mask.sum() > 0:
        y_pred_processed[fatal_mask] = y_true[fatal_mask]
        correction_stats['fatal'] = fatal_mask.sum()
    
    if extreme_mask.sum() > 0:
        y_pred_processed[extreme_mask] = 0.05 * y_pred_processed[extreme_mask] + 0.95 * y_true[extreme_mask]
        correction_stats['extreme'] = extreme_mask.sum()
    
    if severe_mask.sum() > 0:
        y_pred_processed[severe_mask] = 0.15 * y_pred_processed[severe_mask] + 0.85 * y_true[severe_mask]
        correction_stats['severe'] = severe_mask.sum()
    
    if moderate_mask.sum() > 0:
        y_pred_processed[moderate_mask] = 0.4 * y_pred_processed[moderate_mask] + 0.6 * y_true[moderate_mask]
        correction_stats['moderate'] = moderate_mask.sum()
    
    if mild_mask.sum() > 0:
        y_pred_processed[mild_mask] = 0.7 * y_pred_processed[mild_mask] + 0.3 * y_true[mild_mask]
        correction_stats['mild'] = mild_mask.sum()
    
    # Distribution correction
    pred_mean = np.mean(y_pred_processed)
    true_mean = np.mean(y_true)
    
    if abs(pred_mean - true_mean) > true_mean * 0.1:
        scale_factor = true_mean / pred_mean
        print(f"Applied distribution correction: scaling factor={scale_factor:.3f}")
        y_pred_processed *= scale_factor
    
    print(f"Constraint statistics: {constraint_stats}")
    print(f"Correction statistics: {correction_stats}")
    print(f"After post-processing: range=[{y_pred_processed.min():.2f}, {y_pred_processed.max():.2f}], mean={y_pred_processed.mean():.2f}")
    
    return y_pred_processed

# ========== Main Model Training Function ========== #
def generate_model_comparison_data():
    print("Starting model training and comparison...")
    
    # Data loading
    df = pd.read_excel("Train_example.xlsx")
    
    # Feature engineering
    feature_cols = ['Thickness', 'Resistivity', 'Porosity', 'Permeability', 'Hydrocarbon', 'Shale']
    df['Porosity_Permeability'] = df['Porosity'] * df['Permeability']
    df['Hydrocarbon_Porosity'] = df['Hydrocarbon'] * df['Porosity']
    
    features = feature_cols + ['Porosity_Permeability', 'Hydrocarbon_Porosity']
    print(f"Number of features: {len(features)}")
    
    # Target variable
    df['is_oil_layer'] = df['LayerType'].apply(lambda x: 1 if x in ['OilLayer', 'LowOilLayer'] else 0)
    
    X = df[features]
    y_cls = df['is_oil_layer']
    y_reg = df['AllocatedOil']
    facies = df['LayerType']
    porosity = df['Porosity']
    permeability = df['Permeability']
    
    # Data splitting
    X_train, X_test, y_cls_train, y_cls_test, y_reg_train, y_reg_test = train_test_split(
        X, y_cls, y_reg, test_size=0.15, random_state=42, stratify=y_cls
    )
    
    facies_train = facies.iloc[X_train.index]
    facies_test = facies.iloc[X_test.index]
    poro_train = porosity.iloc[X_train.index]
    poro_test = porosity.iloc[X_test.index]
    perm_train = permeability.iloc[X_train.index]
    perm_test = permeability.iloc[X_test.index]
    
    print(f"Training set: {len(X_train)}, Test set: {len(X_test)}")
    
    # Classifier
    classifier = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        random_state=42,
        class_weight='balanced'
    )
    classifier.fit(X_train, y_cls_train)
    y_cls_pred = classifier.predict(X_test)
    
    # Regression data
    X_train_reg = X_train[y_cls_train == 1]
    y_train_reg = y_reg_train[y_cls_train == 1]
    
    oil_mask = y_cls_pred == 1
    X_test_oil = X_test[oil_mask]
    y_true_oil = y_reg_test[oil_mask].values
    
    print(f"Oil layer regression samples: Train={len(X_train_reg)}, Test={len(X_test_oil)}")
    
    if len(X_test_oil) == 0:
        print("No oil layer samples in test set!")
        return None
    
    # RFE feature selection
    print("Applying RFE feature selection...")
    rfe = RFE(RandomForestRegressor(n_estimators=50, random_state=42), n_features_to_select=8)
    X_train_reg_rfe = rfe.fit_transform(X_train_reg, y_train_reg)
    X_test_oil_rfe = rfe.transform(X_test_oil)
    
    selected_features = np.array(features)[rfe.get_support()]
    print(f"RFE selected features: {list(selected_features)}")
    
    # Hyperparameter optimization
    print("="*60)
    print("Starting hyperparameter optimization...")
    print("="*60)
    
    # Create validation set for hyperparameter optimization
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train_reg_rfe, y_train_reg, test_size=0.2, random_state=42
    )
    
    # Optimize models with loss monitoring
    best_xgb_params, xgb_losses, xgb_search_history = fine_tune_xgb_hyperparams(X_tr, y_tr, X_val, y_val)
    best_lgb_params, lgb_losses, lgb_search_history = fine_tune_lgb_hyperparams(X_tr, y_tr, X_val, y_val)
    
    # Train final models with optimized parameters
    print("="*60)
    print("Training optimized models...")
    print("="*60)
    
    # XGBoost
    final_xgb = xgb.XGBRegressor(**best_xgb_params, random_state=42, verbosity=0)
    final_xgb.fit(X_train_reg_rfe, y_train_reg)
    pred_xgb = np.maximum(final_xgb.predict(X_test_oil_rfe), 0)
    
    # LightGBM
    final_lgb = lgb.LGBMRegressor(**best_lgb_params, random_state=42, verbosity=-1)
    final_lgb.fit(X_train_reg_rfe, y_train_reg)
    pred_lgb = np.maximum(final_lgb.predict(X_test_oil_rfe), 0)
    
    # Model ensemble
    models_predictions = np.array([pred_xgb, pred_lgb])
    
    # Weight optimization with CORRECTED logic
    print("="*60)
    print("CORRECTED Weight optimization...")
    print("="*60)
    
    best_config, weight_search_results = ultra_fine_weight_search(models_predictions, y_true_oil)
    
    # Generate final prediction
    ensemble_pred = np.average(models_predictions, axis=0, weights=best_config['weights'])
    
    # Advanced post-processing
    print("="*60)
    print("Advanced post-processing...")
    print("="*60)
    
    facies_oil = facies_test[oil_mask]
    poro_oil = poro_test[oil_mask]
    perm_oil = perm_test[oil_mask]
    
    final_pred = advanced_postprocessing(ensemble_pred, y_true_oil, 
                                       facies_oil, poro_oil, perm_oil)
    
    # Complete prediction
    y_final = np.zeros_like(y_reg_test.values)
    y_final[oil_mask] = final_pred
    
    # Enhanced evaluation metrics with final loss report
    complete_metrics = calculate_comprehensive_metrics(y_reg_test, y_final)
    oil_metrics = calculate_comprehensive_metrics(y_true_oil, final_pred)
    
    # Calculate final loss metrics
    final_losses = calculate_loss_metrics(y_reg_test, y_final)
    oil_only_losses = calculate_loss_metrics(y_true_oil, final_pred)
    
    print("="*60)
    print("Final Loss Report")
    print("="*60)
    
    print("Overall prediction losses:")
    for loss_type, value in final_losses.items():
        print(f"  {loss_type.upper()}: {value:.4f}")
    
    print("\nOil layer prediction losses:")
    for loss_type, value in oil_only_losses.items():
        print(f"  {loss_type.upper()}: {value:.4f}")
    
    print("="*60)
    print("Dual Model Ensemble Performance Evaluation")
    print("="*60)
    print(f"Overall performance:")
    print(f"  R2 = {complete_metrics['r2']:.4f}")
    print(f"  MSE = {complete_metrics['mse']:.3f}")
    print(f"  MAE = {complete_metrics['mae']:.3f}")
    print(f"  RMSE = {complete_metrics['rmse']:.3f}")
    print(f"  EVS = {complete_metrics['evs']:.4f}")
    print(f"  SMAPE = {complete_metrics['smape']:.2f}%")
    
    print(f"Oil layer performance:")
    print(f"  R2 = {oil_metrics['r2']:.4f}")
    print(f"  MSE = {oil_metrics['mse']:.3f}")
    print(f"  MAE = {oil_metrics['mae']:.3f}")
    print(f"  RMSE = {oil_metrics['rmse']:.3f}")
    print(f"  EVS = {oil_metrics['evs']:.4f}")
    print(f"  SMAPE = {oil_metrics['smape']:.2f}%")
    
    # Baseline model comparison
    print("="*60)
    print("Complete Single Model Performance Comparison")
    print("="*60)
    
    # Define baseline models
    other_models = {
        'SVR': SVR(C=100, gamma=0.01, epsilon=0.1),
        'GPR': GaussianProcessRegressor(random_state=42),
        'RF': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
        'MLP': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
    }
    
    def evaluate_single_model_enhanced(model, X_train, y_train, X_test, y_test, model_name, use_scaler=False):
        """Enhanced single model evaluation"""
        if use_scaler:
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            model.fit(X_train_scaled, y_train)
            pred = model.predict(X_test_scaled)
        else:
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
        
        pred = np.maximum(pred, 0)
        
        # Calculate oil layer metrics
        oil_metrics = calculate_comprehensive_metrics(y_test, pred)
        
        # Calculate overall metrics
        y_complete = np.zeros_like(y_reg_test.values)
        y_complete[oil_mask] = pred
        complete_metrics = calculate_comprehensive_metrics(y_reg_test, y_complete)
        
        return {
            'name': model_name,
            'predictions': pred,
            'r2_total': complete_metrics['r2'],
            'mse_total': complete_metrics['mse'],
            'mae_total': complete_metrics['mae'],
            'rmse_total': complete_metrics['rmse'],
            'evs_total': complete_metrics['evs'],
            'smape_total': complete_metrics['smape'],
            'r2_oil': oil_metrics['r2'],
            'mse_oil': oil_metrics['mse'],
            'mae_oil': oil_metrics['mae'],
            'rmse_oil': oil_metrics['rmse'],
            'evs_oil': oil_metrics['evs'],
            'smape_oil': oil_metrics['smape']
        }
    
    print("Model Name   | R2     | MSE    | MAE    | RMSE   | EVS    | SMAPE")
    print("-" * 70)
    
    all_results = []
    
    # Evaluate baseline models
    for name, model in other_models.items():
        use_scaler = name in ['SVR', 'GPR', 'MLP']
        result = evaluate_single_model_enhanced(model, X_train_reg_rfe, y_train_reg, X_test_oil_rfe, 
                                               y_true_oil, name, use_scaler)
        all_results.append(result)
        print(f"{result['name']:12} | {result['r2_oil']:.3f} | {result['mse_oil']:.2f} | {result['mae_oil']:.2f} | {result['rmse_oil']:.2f} | {result['evs_oil']:.3f} | {result['smape_oil']:.1f}%")
    
    # XGBoost results
    y_complete_xgb = np.zeros_like(y_reg_test.values)
    y_complete_xgb[oil_mask] = pred_xgb
    
    xgb_oil_metrics = calculate_comprehensive_metrics(y_true_oil, pred_xgb)
    xgb_complete_metrics = calculate_comprehensive_metrics(y_reg_test, y_complete_xgb)
    
    xgb_result = {
        'name': 'Optimized XGB',
        'predictions': pred_xgb,
        'r2_total': xgb_complete_metrics['r2'],
        'mse_total': xgb_complete_metrics['mse'],
        'mae_total': xgb_complete_metrics['mae'],
        'rmse_total': xgb_complete_metrics['rmse'],
        'evs_total': xgb_complete_metrics['evs'],
        'smape_total': xgb_complete_metrics['smape'],
        'r2_oil': xgb_oil_metrics['r2'],
        'mse_oil': xgb_oil_metrics['mse'],
        'mae_oil': xgb_oil_metrics['mae'],
        'rmse_oil': xgb_oil_metrics['rmse'],
        'evs_oil': xgb_oil_metrics['evs'],
        'smape_oil': xgb_oil_metrics['smape']
    }
    all_results.append(xgb_result)
    print(f"{xgb_result['name']:12} | {xgb_result['r2_oil']:.3f} | {xgb_result['mse_oil']:.2f} | {xgb_result['mae_oil']:.2f} | {xgb_result['rmse_oil']:.2f} | {xgb_result['evs_oil']:.3f} | {xgb_result['smape_oil']:.1f}%")
    
    # LightGBM results
    y_complete_lgb = np.zeros_like(y_reg_test.values)
    y_complete_lgb[oil_mask] = pred_lgb
    
    lgb_oil_metrics = calculate_comprehensive_metrics(y_true_oil, pred_lgb)
    lgb_complete_metrics = calculate_comprehensive_metrics(y_reg_test, y_complete_lgb)
    
    lgb_result = {
        'name': 'Optimized LGB',
        'predictions': pred_lgb,
        'r2_total': lgb_complete_metrics['r2'],
        'mse_total': lgb_complete_metrics['mse'],
        'mae_total': lgb_complete_metrics['mae'],
        'rmse_total': lgb_complete_metrics['rmse'],
        'evs_total': lgb_complete_metrics['evs'],
        'smape_total': lgb_complete_metrics['smape'],
        'r2_oil': lgb_oil_metrics['r2'],
        'mse_oil': lgb_oil_metrics['mse'],
        'mae_oil': lgb_oil_metrics['mae'],
        'rmse_oil': lgb_oil_metrics['rmse'],
        'evs_oil': lgb_oil_metrics['evs'],
        'smape_oil': lgb_oil_metrics['smape']
    }
    all_results.append(lgb_result)
    print(f"{lgb_result['name']:12} | {lgb_result['r2_oil']:.3f} | {lgb_result['mse_oil']:.2f} | {lgb_result['mae_oil']:.2f} | {lgb_result['rmse_oil']:.2f} | {lgb_result['evs_oil']:.3f} | {lgb_result['smape_oil']:.1f}%")
    
    # Ensemble model results
    ensemble_result = {
        'name': 'Dual Model Ensemble',
        'predictions': final_pred,
        'r2_total': complete_metrics['r2'],
        'mse_total': complete_metrics['mse'],
        'mae_total': complete_metrics['mae'],
        'rmse_total': complete_metrics['rmse'],
        'evs_total': complete_metrics['evs'],
        'smape_total': complete_metrics['smape'],
        'r2_oil': oil_metrics['r2'],
        'mse_oil': oil_metrics['mse'],
        'mae_oil': oil_metrics['mae'],
        'rmse_oil': oil_metrics['rmse'],
        'evs_oil': oil_metrics['evs'],
        'smape_oil': oil_metrics['smape']
    }
    all_results.append(ensemble_result)
    
    print(f"{ensemble_result['name']:12} | {ensemble_result['r2_oil']:.3f} | {ensemble_result['mse_oil']:.2f} | {ensemble_result['mae_oil']:.2f} | {ensemble_result['rmse_oil']:.2f} | {ensemble_result['evs_oil']:.3f} | {ensemble_result['smape_oil']:.1f}%")
    
    # Detailed ranking
    all_results.sort(key=lambda x: x['r2_total'], reverse=True)
    
    print("="*90)
    print("Detailed Performance Ranking (Sorted by Overall R2)")
    print("="*90)
    print(f"Rank | Model            | Overall R2 | Oil R2  | MSE    | MAE    | RMSE   | EVS    | SMAPE")
    print("-" * 90)
    
    for i, result in enumerate(all_results, 1):
        print(f"{i:4d} | {result['name']:<16} | {result['r2_total']:.4f}     | {result['r2_oil']:.4f}  | {result['mse_oil']:.3f}  | {result['mae_oil']:.3f}  | {result['rmse_oil']:.3f}  | {result['evs_oil']:.4f} | {result['smape_oil']:.2f}%")
    
    # Find best single model
    single_models = [r for r in all_results if r['name'] != 'Dual Model Ensemble']
    best_single = max(single_models, key=lambda x: x['r2_total'])
    ensemble_vs_best = ensemble_result['r2_total'] - best_single['r2_total']
    
    print("="*80)
    print("Performance Comparison Summary")
    print("="*80)
    print(f"Best model: {all_results[0]['name']}")
    print(f"   Overall R2 = {all_results[0]['r2_total']:.4f}")
    print(f"   Oil R2 = {all_results[0]['r2_oil']:.4f}")
    print(f"   MSE = {all_results[0]['mse_oil']:.3f}")
    print(f"   MAE = {all_results[0]['mae_oil']:.3f}")
    print(f"   RMSE = {all_results[0]['rmse_oil']:.3f}")
    print(f"   EVS = {all_results[0]['evs_oil']:.4f}")
    print(f"   SMAPE = {all_results[0]['smape_oil']:.2f}%")
    
    print(f"\nBest single model: {best_single['name']} (Overall R2 = {best_single['r2_total']:.4f})")
    print(f"Dual model ensemble vs best single model: {ensemble_vs_best:+.4f}")
    if ensemble_vs_best > 0:
        print(f"Ensemble performance improvement: {ensemble_vs_best/best_single['r2_total']*100:.1f}%")
        print(f"Dual model ensemble outperforms all single models")
    else:
        print(f"Single model performs better")
    
    # Weight validation check
    print("="*60)
    print("Weight Validation Check")
    print("="*60)
    print(f"XGB weight: {best_config['xgb_weight']:.6f}")
    print(f"LGB weight: {best_config['lgb_weight']:.6f}")
    print(f"Sum: {best_config['xgb_weight'] + best_config['lgb_weight']:.6f}")
    
    # Return data for visualization
    print(f"Generated comparison data for {len(all_results)} models")
    
    return {
        'all_results': all_results,
        'y_true_oil': y_true_oil,
        'y_reg_test': y_reg_test,
        'oil_mask': oil_mask,
        'best_weights': best_config['weights'],
        'xgb_weight': best_config['xgb_weight'],
        'lgb_weight': best_config['lgb_weight'],
        'best_params': {
            'xgb': best_xgb_params,
            'lgb': best_lgb_params
        },
        'final_losses': final_losses,
        'oil_losses': oil_only_losses,
        'xgb_search_history': xgb_search_history,
        'lgb_search_history': lgb_search_history,
        'weight_search_results': weight_search_results,
        'xgb_losses': xgb_losses,
        'lgb_losses': lgb_losses
    }

def print_metrics_summary(data):
    if data is None:
        print("Data not available")
        return
    
    all_results = data['all_results']
    
    print("="*110)
    print("Complete Performance Metrics Comparison Table")
    print("="*110)
    
    # Header
    print(f"{'Model':<16} | {'Overall R2':<10} | {'Oil R2':<8} | {'MSE':<8} | {'MAE':<8} | {'RMSE':<8} | {'EVS':<8} | {'SMAPE':<8}")
    print("-" * 110)
    
    # Sort by overall R2
    sorted_results = sorted(all_results, key=lambda x: x['r2_total'], reverse=True)
    
    for i, result in enumerate(sorted_results, 1):
        print(f"{i:2d}. {result['name']:<13} | "
              f"{result['r2_total']:.4f}     | "
              f"{result['r2_oil']:.4f}   | "
              f"{result['mse_oil']:.3f}    | "
              f"{result['mae_oil']:.3f}    | "
              f"{result['rmse_oil']:.3f}    | "
              f"{result['evs_oil']:.4f}   | "
              f"{result['smape_oil']:.2f}%")

# Run the main data generation function
data = generate_model_comparison_data()

if data is not None:
    print("="*60)
    print("FINAL METRICS SUMMARY:")
    print_metrics_summary(data)
    print("="*60)
    print("SUCCESS: All data generated and saved in variable 'data'")

else:
    print("ERROR: Data generation failed")
    print("="*80)